## Import Required Libraries


In [24]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
import os
from glob import glob
import pylandstats as pls
from rasterstats import zonal_stats
import warnings
import rasterio as rio
import rasterio.mask
from rasterio.warp import calculate_default_transform, reproject, Resampling
import argparse
from tqdm import tqdm

# Suppress warnings to keep output clean
warnings.filterwarnings('ignore')

## Define File and Directory Paths

In [25]:
# Main directory containing raw raster and vector datasets
MAIN_DATA_DIR = r'/beegfs/halder/DATA'

# GitHub-linked project directory where processed data and results are stored
PROJECT_DATA_DIR = r'/beegfs/halder/GITHUB/Landscape-Analysis/data'

# Temporary directory used for storing intermediate files
TEMP_DIR = os.path.join(PROJECT_DATA_DIR, 'TEMP')

## Load Hexagonal Grid (5 km² resolution) for Germany

In [26]:
EPSG = 25832  # Use ETRS89 / UTM Zone 32N as the projection (suitable for Germany)

# Path to grid shapefile
GRID_PATH = os.path.join(PROJECT_DATA_DIR, 'VECTOR', f'DE_Hexbins_10sqkm_EPSG_{EPSG}.shp')

# Load grid as a GeoDataFrame and retain relevant columns
grids_gdf = gpd.read_file(GRID_PATH)
grids_gdf = grids_gdf[['id', 'geometry']]
grids_gdf['id'] = grids_gdf['id'].astype(int)

print('Successfully read the grids!')

Successfully read the grids!


## Compute Crop Metrics Using PyLandStats


In [ ]:
out_dir = os.path.join(PROJECT_DATA_DIR, 'OUTPUT', 'Landscape_Metrics')
os.makedirs(out_dir, exist_ok=True)

# argparse setup
parser = argparse.ArgumentParser(description="Process crop landscape metrics for a given year.")
parser.add_argument('--year', type=int, required=True, help="Year of the crop type raster (e.g., 2017)")
args = parser.parse_args()

year = args.year

# Path to crop type raster (10 m resolution)
crop_file_path = os.path.join(MAIN_DATA_DIR, 'DE_Crop_Types_2017_2021', f'DE_Crop_Type_{year}.tif')
reprojected_raster_path = os.path.join(TEMP_DIR, f'DE_Crop_Type_{year}_EPSG_{EPSG}.tif')

# Target CRS (Coordinate Reference System)
dst_crs = f'EPSG:{EPSG}'

# Open source raster
with rio.open(crop_file_path) as src:
    transform, width, height = calculate_default_transform(
        src.crs, dst_crs, src.width, src.height, *src.bounds)
    
    kwargs = src.meta.copy()
    kwargs.update({
        'crs': dst_crs,
        'transform': transform,
        'width': width,
        'height': height
    })

    # Write reprojected raster
    with rio.open(reprojected_raster_path, 'w', **kwargs) as dst:
        for i in range(1, src.count + 1):
            reproject(
                source=rio.band(src, i),
                destination=rio.band(dst, i),
                src_transform=src.transform,
                src_crs=src.crs,
                dst_transform=transform,
                dst_crs=dst_crs,
                resampling=Resampling.nearest
            )
            
print('Raster Saved Successfully!')

# Collect valid zone indices
valid_indices = []

with rio.open(reprojected_raster_path) as src:
    for idx, row in tqdm(grids_gdf.iterrows(), total=len(grids_gdf)):
        try:
            mask, transform = rio.mask.mask(src, [row['geometry']], crop=True)
            unique_vals = np.unique(mask)
            if len(unique_vals) <= 1:  # Only background or no-data
                print(f"Zone {row['id']} skipped: {unique_vals}")
                continue
            valid_indices.append(idx)
        except Exception as e:
            print(f"Zone {row['id']} caused error: {e}")
            continue

# Filter the GeoDataFrame
grids_gdf = grids_gdf.loc[valid_indices].copy()

# Create ZonalAnalysis object for computing crop metrics per grid zone
za = pls.ZonalAnalysis(
    reprojected_raster_path,  # Use the reprojected raster
    zones=grids_gdf,
    zone_index='id',
    neighborhood_rule=8       # 8-neighbor connectivity for landscape pattern analysis
)

# # Compute class-level metrics (per crop type) for each zone
# class_metrics_df = za.compute_class_metrics_df().reset_index()
# class_metrics_df.to_csv(os.path.join(out_dir, f'crop_class_metrics_{year}.csv'), index=False)

# Compute landscape-level metrics (overall structure) for each zone
landscape_metrics_df = za.compute_landscape_metrics_df().reset_index()
# landscape_metrics_df.to_csv(os.path.join(out_dir, f'crop_landscape_metrics_{year}.csv'), index=False)

print('Crop metrics computation complete!')

Raster Saved Successfully!


  1%|          | 41/4430 [00:06<05:22, 13.61it/s]

Zone 1456 skipped: [0]


  3%|▎         | 137/4430 [00:14<03:57, 18.08it/s]

Zone 1473 skipped: [0]


...

Zone 1645 skipped: [0]


 95%|█████████▌| 4212/4430 [04:37<00:10, 20.77it/s]

Zone 1811 skipped: [0]


100%|██████████| 4430/4430 [04:51<00:00, 15.22it/s]


[######                                  ] | 16% Completed | 24m 33ss


## Compute Crop Intensity using Rasterstats

In [27]:
out_dir = os.path.join(PROJECT_DATA_DIR, 'OUTPUT', f'Landscape_Metrics_{DISTANCE}KM')
os.makedirs(out_dir, exist_ok=True)

# Path to crop mask
crop_file_path = os.path.join(MAIN_DATA_DIR, 'DE_Crop_Types_2017_2021', 'wheat_mask_combined.tif')
reprojected_raster_path = os.path.join(TEMP_DIR, f'wheat_mask_combined_{DISTANCE}.tif')

# Target CRS (Coordinate Reference System)
dst_crs = f'EPSG:{EPSG}'

# Open source raster
with rio.open(crop_file_path) as src:
    transform, width, height = calculate_default_transform(
        src.crs, dst_crs, src.width, src.height, *src.bounds)
    
    kwargs = src.meta.copy()
    kwargs.update({
        'crs': dst_crs,
        'transform': transform,
        'width': width,
        'height': height
    })

    # Write reprojected raster
    with rio.open(reprojected_raster_path, 'w', **kwargs) as dst:
        for i in range(1, src.count + 1):
            reproject(
                source=rio.band(src, i),
                destination=rio.band(dst, i),
                src_transform=src.transform,
                src_crs=src.crs,
                dst_transform=transform,
                dst_crs=dst_crs,
                resampling=Resampling.nearest
            )
            
print('Raster Saved Successfully!')

# Collect valid zone indices
valid_indices = []

with rio.open(reprojected_raster_path) as src:
    for idx, row in tqdm(grids_gdf.iterrows(), total=len(grids_gdf)):
        try:
            mask, transform = rio.mask.mask(src, [row['geometry']], crop=True)
            unique_vals = np.unique(mask)
            if len(unique_vals) <= 1:  # Only background or no-data
                print(f"Zone {row['id']} skipped: {unique_vals}")
                continue
            valid_indices.append(idx)
        except Exception as e:
            print(f"Zone {row['id']} caused error: {e}")
            continue

# Filter the GeoDataFrame
grids_gdf = grids_gdf.loc[valid_indices].copy()

# Calculate zonal stats
stats = zonal_stats(grids_gdf, reprojected_raster_path, stats=["mean"])

# Add results to GeoDataFrame
zones_stats = grids_gdf.copy()
for key in stats[0].keys():
    zones_stats[key] = [s[key] for s in stats]

zones_stats.rename(columns={"mean": 'crop_intensity'}, inplace=True)
zones_stats['normalized_crop_intensity'] = zones_stats['crop_intensity'] / 5
zones_stats = zones_stats[['id', 'geometry', 'crop_intensity', 'normalized_crop_intensity']]

# Save the data
zones_stats.to_csv(os.path.join(out_dir, f'crop_intensity.csv'), index=False)
print('Crop intensity computation complete!')

Raster Saved Successfully!


  1%|          | 41/4430 [00:06<05:26, 13.43it/s]

Zone 1456 skipped: [0]


  3%|▎         | 137/4430 [00:13<02:42, 26.42it/s]

Zone 1493 skipped: [0]
Zone 1473 skipped: [0]


...

Zone 1233 skipped: [0]


100%|█████████▉| 4424/4430 [01:54<00:00, 43.93it/s]

Zone 1059 skipped: [0]
Zone 1145 skipped: [0]


100%|██████████| 4430/4430 [01:55<00:00, 38.49it/s]


Crop intensity computation complete!
